In [1]:
%config IPCompleter.greedy=True

In [14]:
import sys
import os
import pandas as pd
sys.path.append("../../../../") # relative path to module toolkit

In [15]:
from keras.models import Model, load_model
from keras_preprocessing.image import ImageDataGenerator
from toolkit import getLabelsFromDir, plot_confusion_matrix
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

ImportError: cannot import name 'getLabelsFromDir'

In [ ]:
# setup
batch_size = 1
val_dir = "../../../../images/backgrounds_beetles_ratio_416/backgrounds"
val_images = 3111
datasetDir = "images_species_shuffled_aspect_416/"
saved_model = '../highest_val_acc.h5'
results_file = "results_noBeetles.csv"

In [ ]:
labels = getLabelsFromDir(val_dir)
val_datagen = ImageDataGenerator(rescale=1./255.)
validation_steps = int(val_images/batch_size) 

In [ ]:
model = load_model(saved_model)

In [ ]:
val_generator = val_datagen.flow_from_directory(val_dir,
                                                    classes=labels,
                                                    class_mode="categorical",
                                                    batch_size=batch_size,
                                                    color_mode='rgb',
                                                    target_size=(416, 416),
                                                    shuffle=False,
                                                    seed=1)

In [ ]:
Y_pred = model.predict_generator(val_generator, steps=validation_steps, verbose=1)

In [ ]:
y_pred = np.argmax(Y_pred, axis=1)
y_pred3 = [y[-3:] for y in np.argsort(Y_pred, axis=1)] 

In [ ]:
print(classification_report(val_generator.classes, y_pred, target_names=labels))

In [ ]:
y_true = val_generator.classes
matplotlib.rcParams['figure.figsize'] = [15, 15]
plot_confusion_matrix(y_true, y_pred, np.array(labels), normalize=True)

In [ ]:
labels = (val_generator.class_indices) # dict of label:index
labels = dict((i,l) for l,i in labels.items()) # dict of index:label

In [ ]:
# create pd.dataframe and save predictions to .csv
filenames=[datasetDir + name for name in val_generator.filenames]
y_pred_labels = [labels[y] for y in y_pred]
y_true_labels = [labels[y] for y in y_true]
y_pred3_labels = []
for i,top3 in enumerate(y_pred3):
    l = []
    for top in top3:
        l.append((labels[top], Y_pred[i][top]))
    y_pred3_labels.append(l)

results=pd.DataFrame({"Filename":filenames,
                      "Label":y_true_labels,
                      "Prediction1":y_pred_labels,
                      "Prediction3":y_pred3_labels})
results.to_csv(results_file,index=True)